### Download Flowers Dataset

In [0]:
#You can download the data manually as well instead of using 'wget'
!wget http://download.tensorflow.org/example_images/flower_photos.tgz --quiet

In [0]:
#Read the dataset
import tarfile
dataset = tarfile.open('flower_photos.tgz')

In [0]:
#We will build a pandas dataset
import pandas as pd
df = pd.DataFrame(columns=['class','dir','image_file'])

In [0]:
#Run through tarfile members 
for name in dataset.getnames():
    
    tar_mem = dataset.getmember(name)
    
    #Check if it is a file
    if(tar_mem.isfile() and name.endswith('.jpg')):
        #Build directory and class info
        im_dir = name[0:name.rfind('/')]
        im_class = im_dir[im_dir.rfind('/')+1:]
        #Add record to the dataframe
        df.loc[df.shape[0]] = [im_class, im_dir, name]

In [6]:
#Check contents of the dataframe
print('Total images: ', df.shape[0])
#df.head()

Total images:  3670


In [0]:
#extract data
dataset.extractall(path='data/')

Create Training & Test Dataset

In [0]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [0]:
train_df.to_csv('data/flower_photos/train.csv',index=False)
test_df.to_csv('data/flower_photos/test.csv', index=False)

### Load tensorflow

In [0]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(42)

### Read training and test data

In [0]:
import pandas as pd
train_df = pd.read_csv('data/flower_photos/train.csv')
test_df = pd.read_csv('data/flower_photos/test.csv')

In [13]:
#Get class names
class_names = train_df['class'].unique().tolist()
print('Flower classes: ', class_names)

Flower classes:  ['tulips', 'daisy', 'sunflowers', 'dandelion', 'roses']


In [0]:
#Define some parameters
img_size = 60
img_depth = 3  

### Build batch generator

In [0]:
import numpy as np

In [16]:
!ls -l data/flower_photos

total 900
drwx------ 2 270850 5000  36864 Feb 10  2016 daisy
drwx------ 2 270850 5000  53248 Feb 10  2016 dandelion
-rw-r----- 1 270850 5000 418049 Feb  9  2016 LICENSE.txt
drwx------ 2 270850 5000  36864 Feb 10  2016 roses
drwx------ 2 270850 5000  36864 Feb 10  2016 sunflowers
-rw-r--r-- 1 root   root  58488 Sep 21 04:42 test.csv
-rw-r--r-- 1 root   root 234608 Sep 21 04:42 train.csv
drwx------ 2 270850 5000  36864 Feb 10  2016 tulips


#### Batch Generation using Keras ImageDataGenerator

In [17]:
#Create an Image Generator
img_generator= tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                                 width_shift_range=0.2,
                                                                 height_shift_range=0.2,
                                                                 horizontal_flip=True, 
                                                                 validation_split=0.2)

#Build train generator
train_generator = img_generator.flow_from_directory('data/flower_photos',
                                                    target_size=(img_size, img_size),
                                                    subset='training',
                                                    batch_size=64)

#Build test generator
test_generator = img_generator.flow_from_directory('data/flower_photos',
                                                    target_size=(img_size, img_size),                                                   
                                                    subset='validation',                                                   
                                                   batch_size=64)

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [0]:
def batch_generator(df, batchsize=32, train_mode=True):   
    
    img_generator= tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                                   width_shift_range=0.2,
                                                                   height_shift_range=0.2,
                                                                   horizontal_flip=True)
    
    while True:       
        
        
        #Generate random numbers to pick images from dataset
        batch_nums = np.random.randint(0,df.shape[0],batchsize)
        
        #Initialize batch images array
        batch_images = np.zeros((batchsize,img_size, img_size,img_depth))
        
        #Initiate batch label array
        batch_labels = np.zeros((batchsize, len(class_names)))
        
        for i in range(batchsize):
            
            #Load image
            flower_image =  tf.keras.preprocessing.image.load_img('data/' + 
                                                                  df.loc[batch_nums[i]]['image_file'],
                                                                  target_size=(img_size, img_size))
            #Convert to array
            flower_image = tf.keras.preprocessing.image.img_to_array(flower_image)
            
            if(train_mode):
                #Apply transform
                flower_image =  img_generator.random_transform(flower_image)
            
            
            #Get the class
            img_class = df.loc[batch_nums[i]]['class']
            #Conver class to number
            img_class = class_names.index(img_class)
            #Convert class to one hot encoding
            img_class = tf.keras.utils.to_categorical(img_class, num_classes=len(class_names))
            
            #Update batch images and class arrays
            batch_images[i] = flower_image
            batch_labels[i] = img_class        
        
        yield batch_images, batch_labels          

### Build CNN Model

In [18]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=(img_size,img_size,3,)))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))

#Output layer
model.add(tf.keras.layers.Dense(len(class_names), activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 60, 60, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 58, 58, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 28, 28, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               6422656   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0

### Train the model

In [22]:
model.fit_generator(train_generator, 
                          epochs=50,
                          validation_data=test_generator)

Epoch 1/50
46/46 [==============================] - 15s 328ms/step - loss: 1.9195 - acc: 0.3175 - val_loss: 1.7211 - val_acc: 0.3529
Epoch 2/50
46/46 [==============================] - 10s 227ms/step - loss: 1.2749 - acc: 0.4719 - val_loss: 1.2705 - val_acc: 0.5007
Epoch 3/50
46/46 [==============================] - 10s 227ms/step - loss: 1.1736 - acc: 0.5230 - val_loss: 1.1764 - val_acc: 0.5075
Epoch 4/50
46/46 [==============================] - 11s 231ms/step - loss: 1.0945 - acc: 0.5747 - val_loss: 1.0514 - val_acc: 0.5841
Epoch 5/50
46/46 [==============================] - 10s 228ms/step - loss: 1.0369 - acc: 0.5999 - val_loss: 1.0051 - val_acc: 0.6170
Epoch 6/50
46/46 [==============================] - 11s 229ms/step - loss: 0.9706 - acc: 0.6267 - val_loss: 0.9778 - val_acc: 0.6252
Epoch 7/50
46/46 [==============================] - 11s 231ms/step - loss: 0.9436 - acc: 0.6359 - val_loss: 0.9379 - val_acc: 0.6402
Epoch 8/50
46/46 [==============================] - 10s 227ms/step - 

In [0]:
model.save('flowers.h5')